# MATH 210 Introduction to Mathematical Computing

## November 25, 2020

* How does `scipy.solve` work?
* LU factorization

## How does scipy.linalg.solve work?

Look at the documentation for [scipy.linalg.solve](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve.html#scipy.linalg.solve) and we see that `solve` uses [LAPACK](https://en.wikipedia.org/wiki/LAPACK). In particular, it uses the [LU factorization](https://www.netlib.org/lapack/lug/node38.html) to solve a system of equations $A \mathbf{x} = \mathbf{b}$:

1. Compute $A = LU$ (more generally $A = PLU$)
2. Solve lower triangular system $L \mathbf{y} = \mathbf{b}$
3. Solve upper triangular system $U \mathbf{x} = \mathbf{y}$

Remember $A=LU$ only exists if we can reduce $A$ without switching rows.

## LU Factorization

Write a function called `lu` which takes $A$ and returns a tuple of NumPy arrays `(L,U)` where $A = LU$, or if $LU$ factorization does not exist return `None`.

In [1]:
import numpy as np

In [2]:
def lu(A):
    # Return None if A is not a square matrix
    if A.shape[0] != A.shape[1]:
        return None
    N = A.shape[0]
    L = np.eye(N)
    U = A
    # n across the columns and m down the rows
    for n in range(0,N-1):
        for m in range(n+1,N):
            # Return None if the pivot entry U[n,n] is too small
            if abs(U[n,n]) < 1e-10:
                return None
            # Make U[m,n] equal to 0: add -U[m,n]/U[n,n] times row n to row m
            E = np.eye(N)
            E[m,n] = -U[m,n]/U[n,n]
            # Update L = E1^{-1}E2^{-1}...EM^{-1}
            L[m,n] = U[m,n]/U[n,n]
            U = E@U
    return (L,U)

In [3]:
A = np.array([[1,2],[3,4],[5,6]])
result = lu(A)
print(result)

None


In [4]:
A = np.array([[1,2],[3,4]])
L,U = lu(A)
print(' A =\n'); print(A)
print('\n L =\n'); print(L)
print('\n U =\n'); print(U)
print('\nLU =\n'); print(L@U)

 A =

[[1 2]
 [3 4]]

 L =

[[1. 0.]
 [3. 1.]]

 U =

[[ 1.  2.]
 [ 0. -2.]]

LU =

[[1. 2.]
 [3. 4.]]


In [5]:
A = np.random.randint(-5,5,(5,5))
result = lu(A)
# Show result if lu does not return None
if result:
    L,U = result
    print(' A =\n'); print(A)
    print('\n L =\n'); print(L)
    print('\n U =\n'); print(U)
    print('\nLU =\n'); print(L@U)

 A =

[[-3 -5 -3  1 -5]
 [-1 -4  4 -3 -1]
 [-1  0 -5  4 -4]
 [-2  4 -5 -3  3]
 [ 3 -2  0 -3  2]]

 L =

[[  1.           0.           0.           0.           0.        ]
 [  0.33333333   1.           0.           0.           0.        ]
 [  0.33333333  -0.71428571   1.           0.           0.        ]
 [  0.66666667  -3.14285714 -29.66666667   1.           0.        ]
 [ -1.           3.          42.          -1.91666667   1.        ]]

 U =

[[ -3.          -5.          -3.           1.          -5.        ]
 [  0.          -2.33333333   5.          -3.33333333   0.66666667]
 [  0.           0.          -0.42857143   1.28571429  -1.85714286]
 [  0.           0.           0.          24.         -46.66666667]
 [  0.           0.           0.           0.         -16.44444444]]

LU =

[[-3. -5. -3.  1. -5.]
 [-1. -4.  4. -3. -1.]
 [-1.  0. -5.  4. -4.]
 [-2.  4. -5. -3.  3.]
 [ 3. -2.  0. -3.  2.]]


## Triangular system

Write a function called `solve_lower_triangular` which takes input parameters `L` and `b` and returns the solution `y` of the lower triangular system `Ly = b`. In particular, if we have an $N \times N$ lower triangular system:

\begin{array}{ccccccccccc}
l_{0,0}y_0 & & & & & & & & & = & b_0 \\
l_{1,0}y_0 & + & l_{1,1}y_1 & & & & & & & = & b_1 \\
l_{2,0}y_0 & + & l_{2,1}y_1 & + & l_{2,2}y_2 & & & & & = & b_2 \\
& \vdots & & & & \ddots & & & & \vdots & \\
l_{N-1,0}y_0 & + & l_{N-1,1}y_1 & + & l_{N-1,2}y_2 & + & \cdots & + & l_{N-1,N-1}y_{N-1} & = & b_{N-1} \\
\end{array}

Then we solve

\begin{align*}
y_0 &= b_0/l_{0,0} \\
y_1 &= (b_1 - l_{1,0}y_0)/l_{1,1} \\
y_2 &= (b_2 - l_{2,0}y_0 - l_{2,1}y_1)/l_{2,2} \\
& \vdots \\
y_{N-1} &= (b_{N-1} - l_{N-1,0}y_0 - l_{N-1,1}y_1 - l_{N-1,2}y_2 - \cdots - l_{N-1,N-2}y_{N-2})/l_{N-1,N-1} \\
\end{align*}



In [6]:
def solve_lower_triangular(L,b):
    if L.shape[0] != L.shape[1]:
        return None
    N = L.shape[1]
    y = np.zeros((N,1))
    for n in range(0,N):
        y[n] = (b[n] - sum([L[n,k]*y[k] for k in range(0,n+1)]))/L[n,n]
    return y

In [14]:
L = np.array([[1,0],[-2,2]])
b = np.array([[4],[-5]])
y = solve_lower_triangular(L,b)
print('\n L =\n'); print(L)
print('\n b =\n'); print(b)
print('\n y =\n'); print(y)
print('\nLy =\n'); print(L@y)


 L =

[[ 1  0]
 [-2  2]]

 b =

[[ 4]
 [-5]]

 y =

[[4. ]
 [1.5]]

Ly =

[[ 4.]
 [-5.]]


Success!

## Exercise 1

Write a function called `solve_upper_triangular` which takes input parameters `U` and `y` and returns the solution `x` of the upper triangular system `Ux = y`.

## Exercise 2

Combine the functions above to write a function called `solve` which takes input parameters `A` and `b` and returns the solution `x` of the system `Ax = b`. 